# Features extractor

## Read datasets

In [1]:
import pandas as pd
import numpy as np
import datetime
import config as c
from collections import OrderedDict

In [2]:
compute_only_class_a = True
datasets_fake = ["FSF", "INT", "TWT"]
datasets_hum = ["TFP", "E13"]
datasets = datasets_hum# + datasets_fake

In [3]:
users = pd.read_csv(c.folder_datasets + datasets[0] + '/' + c.file_users)
tweets = pd.read_csv(c.folder_datasets + datasets[0] + '/' + c.file_tweets, dtype={"geo": str})
friends = pd.read_csv(c.folder_datasets + datasets[0] + '/' + c.file_friends)
followers = pd.read_csv(c.folder_datasets + datasets[0] + '/' + c.file_followers)
for dataset in datasets[1:]:
    users = pd.concat([users, pd.read_csv(c.folder_datasets + dataset + '/' + c.file_users)])
    tweets = pd.concat([tweets, pd.read_csv(c.folder_datasets + dataset + '/' + c.file_tweets, dtype={"geo": str})])
    friends = pd.concat([friends, pd.read_csv(c.folder_datasets + dataset + '/' + c.file_friends)])
    followers = pd.concat([followers, pd.read_csv(c.folder_datasets + dataset + '/' + c.file_followers)])  

## Compute features

In [4]:
def is_from_web():
    for n in tweets.source.unique():
        for src in ["iphone", "android", "foursquare", "instagram", "web"]:
            if src in n.lower():
                return 1
    return 0

import string
def use_ponctuation(user_id):
    # If one tweets use ponctuation return 1
    user_tweets = tweets.loc[tweets['user_id'] == user['id'], 'text']
    for tweet in user_tweets:
        for c in tweet:
            # See if the char is punctuation.
            if c in string.punctuation:
                return 1
        return 0


In [5]:
import time
import datetime
start_time = time.time()
SAMPLE_SIZE = 100
X = list()
y = list()
features_name = list()
nb_fake_acc =0
# Compute features for each Twitter account
for index, user in users.iterrows():
    acc_feat = OrderedDict()
    #if index > SAMPLE_SIZE:
    #    break
    
    # Class A (Profile)
    #===================
    if int(user['followers_count']) == 0:
        acc_feat["friends/follower_ratio"] = 0
    else :
        acc_feat["friends/follower_ratio"] = int(user['friends_count'])/int(user['followers_count'])
    if "bot" in user['name'] :
        acc_feat["is_bot_in_name"] = 1
    else:
        acc_feat["is_bot_in_name"] = 0
    '''
    acc_feat['has_name'] = int(user['name'] not in ['NULL', 'NaN', '', ' ', pd.np.nan])
    acc_feat['has_image'] = int(user['default_profile_image'] != 1)
    acc_feat['has_address'] = int(user['location'] not in ['NULL', 'NaN', '', ' ', pd.np.nan])
    acc_feat['has_biography'] = int(user['description'] not in ['NULL', 'NaN', '', ' ', pd.np.nan])
    acc_feat['followers_ge_30'] = int(user['followers_count'] >= 30)
    acc_feat['belongs_to_a_list'] = int(user['listed_count'] > 0)
    acc_feat['nb_tweets_ge_50'] = int(tweets.loc[tweets['user_id'] == user['id']].size >= 50)
    acc_feat['url_in_profile'] = int(user['url'] not in ['NULL', 'NaN', '', ' ', pd.np.nan])
    acc_feat['followers_2_times_ge_friends'] = int(2 * user['followers_count'] >= user['friends_count'])
    
    acc_feat['bot_in_biography'] = int(type(user['description']) is str and 'bot' in user['description'].lower())
    acc_feat['ratio_friends_followers_around_100'] = int(user['followers_count'] > 0 and 80.0 <= float(user['friends_count']) / user['followers_count'] >= 120.0)
    acc_feat['duplicate_profile_picture'] = int(users.loc[users['default_profile_image'] == user['default_profile_image']].size > 1)
    
    acc_feat['ratio_friends_followers_ge_50'] = int(user['followers_count'] > 0 and float(user['friends_count']) / user['followers_count'] >= 50)
    acc_feat['default_image_after_2_month'] = int(user['default_profile_image'] == 1 and (datetime.datetime.now() - datetime.datetime.strptime(user['created_at'],'%a %b %d %H:%M:%S +0000 %Y')) > datetime.timedelta(weeks=4)) 
    acc_feat['friends_ge_100'] = int(user['friends_count'] >= 100)
    acc_feat['no_bio'] = int(user['description'] in ['NULL', 'NaN', '', ' ', pd.np.nan])
    acc_feat['no_location'] = int(user['location'] in ['NULL', 'NaN', '', ' ', pd.np.nan])
    acc_feat['no_tweets'] = int(tweets.loc[tweets['user_id'] == user['id']].size == 0)
    
    acc_feat['nb_friends'] = int(user['friends_count'])
    acc_feat['nb_tweets'] = int(tweets.loc[tweets['user_id'] == user['id']].size)
    acc_feat['ratio_friends_followers_square'] = float(user['friends_count']) / pow(user['followers_count'], 2) if user['followers_count'] > 0 else 0
    
    acc_feat['age'] = (datetime.datetime.now() - datetime.datetime.strptime(user['created_at'],'%a %b %d %H:%M:%S +0000 %Y')).total_seconds()
    acc_feat['following_rate'] = float(user['friends_count']) / acc_feat['age']
    '''
    
    # Class B (Timeline)
    #===================
    if not compute_only_class_a:
        #acc_feat['geo_localized'] = int(tweets.loc[tweets['user_id'] == user['id'] & (tweets['geo'] != '')].size)
        #acc_feat['is_favorite'] = int(tweets.loc[tweets['user_id'] == user['id'] & (tweets['favorite_count'] > 0)].size)
        #acc_feat['from_web'] = is_from_web()
        #acc_feat['use_ponctuation'] = use_ponctuation(user['id'])
        
        #acc_feat['same_sentence']
        acc_feat['from_API'] = tweets.loc[tweets['user_id'] == user['id'] & (tweets['source'] != '<a href="http://twitter.com/tweetbutton" rel="nofollow">Tweet Button</a>')].size > 0
        
    
    target = 1 if user['dataset'] in datasets_fake else 0
    nb_fake_acc = nb_fake_acc + target
    
    y.append(target)
    X.append(list(acc_feat.values()))
    features_name = list(acc_feat.keys())
nb_hum_acc = abs(len(y) - nb_fake_acc)
total_time = datetime.timedelta(seconds=time.time() - start_time)
print("Feature computation time : " + str(total_time))

Feature computation time : 0:00:00.171124


In [6]:
print("TOTAL accounts: " + str(users.size))
print("# fake accounts: " + str(nb_fake_acc))
print("# human accounts: " + str(nb_hum_acc))
print("Features name: " + str(list(features_name)))
print(len(X), len(y))
print(X[0], y[0])

TOTAL accounts: 66300
# fake accounts: 0
# human accounts: 1950
Features name: ['friends/follower_ratio', 'is_bot_in_name']
1950 1950
[0.2900098911968348, 0] 1


In [7]:
import pandas as pd
                                                                     
original_df = pd.DataFrame(X)                                        
original_df.to_pickle("hum_features.pkl")